<a href="https://colab.research.google.com/github/nathanbollig/vet-graduate-expectations-survey/blob/main/SVM_WVMA_nontechnical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Veterinary graduate expectations survey

The main comparison is SVM vs. WVMA, derived from `explore.ipynb`. This notebook focuses on the nontechnical questions, which are:
 -   Communication practices (Q13)
 -   Professional and business practices (Q14)
 -   Ethics and professional practices (Q16)

Start by uploading the data into the working directory. Two files are required:

1.   `SVM.xlsx`: SVM graduate expectations survey results
2.   `WVMA.xlsx`: WVMA graduate expectations survey results

In [1]:
! pip install xlsxwriter

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

## Read in SVM data

In [3]:
# Use top row as header and skip second header row
svm = pd.read_excel('SVM.xlsx', header=0, skiprows=lambda x: x in [1])  

In [4]:
#svm.head(3)

In [5]:
# Read in questions from second header row and associate with column names
question_svm = {}

top_rows = pd.read_excel('SVM.xlsx', nrows=2) 

for col in list(top_rows.columns):
    question_svm[col] = top_rows.iloc[0][col]

## Read in WVMA data

In [6]:
# Use top row as header and skip second header row
wvma = pd.read_excel('WVMA.xlsx', header=0, skiprows=lambda x: x in [1])  

# Read in questions from second header row and associate with column names
question_wvma = {}

top_rows_wvma = pd.read_excel('WVMA.xlsx', nrows=2) 

for col in list(top_rows_wvma.columns):
    question_wvma[col] = top_rows_wvma.iloc[0][col]

## Descriptive Analysis

### Note about organization

There are several levels of organization in our interpretation of this data.

 * `Group`: One of the 4 species groups (companion animal, special species, food animal, or equine)
     * `Question`: A group of procedures in a category such as "Medical Procedures" or "Surgical Procedures"
          * `Sub-question`: A particular procedure

We can perform analysis at the sub-question level, or pool upwards to the question or group level. I will do all of this below.





### Ordinal encoding

Below is code to convert responses to ordinal representations and compare the distributions between SVM and WVMA.

Let's encode the expectation response in the following way:

 * 0: No Expectation to Perform Procedure

 * 1: Perform with Assistance (assist with portions of procedure)
 
 * 2: Perform with Direct Supervision (present in room during procedure)

 * 3: Perform with Indirect Supervision (available in building or by phone if needed)

 * 4: Perform Independently

In [7]:
def encode_expectation(response_string):
    if isinstance(response_string, int) == True:
        return response_string
    
    # Encode nan values as -1
    if isinstance(response_string, str) == False:
        if np.isnan(response_string) == True:
            return -1
    
    # Encode string
    s = response_string.lower()
    if s.find('no expectation') > -1:
        return 0
    elif s.find('with assistance') > -1:
        return 1
    elif s.find('indirect supervision') > -1:
        return 3
    elif s.find('direct supervision') > -1:
        return 2
    elif s.find('independently') > -1:
        return 4
    else:
        print(response_string)
        raise ValueError('Expected performance response was not formatted as expected.')

## Reusable code for question analysis

In [8]:
def analyze_question(question_number, filtered_svm_df, filtered_wvma_df, n_subquestions, alpha=0.05, verbose=True):
    """
    Perform an analysis of a given question on a species-filtered dataframe.
    
    Inputs:
        question_number: main question number to analyze
        filtered_svm_df: SVM dataframe filtered to respondants with the desired species area
        filtered_wvma_df: WVMA dataframe filtered to respondants with the desired species area
        n_subquestions: number of subquestions in the main question
        alpha: power level for the statistical test

    Prints a summary of results.

    Outputs:
        table: summary table
        (pooled_stat, pooled_p, pooled_diff_mean): tuple of statistics describing output of Kruskal test on data pooled across subquestions
        svm_data: list of pooled SVM data
        wvma_data: list of pooled WVMA data
        sig_count: number of subquestions with significant difference detected (between SVM and WVMA responses), according to Kruskal test applied at subquestion level

    """

    svm_counts = np.zeros((n_subquestions, 6), dtype=int) # Row for each question, column for empty (-1), 0, 1, 2, 3, and 4 responses
    wvma_counts = np.zeros((n_subquestions, 6), dtype=int) # Row for each question, column for empty (-1), 0, 1, 2, 3, and 4 responses
    rows = []
    svm_pooled = []
    wvma_pooled = []
    sig_count = 0

    for i in range(1, n_subquestions+1):
        qkey = "Q" + str(question_number) + "_" + str(i)
        qstring = question_svm[qkey].split('-')[2] # could refer to questions_svm or questions_wvma

        # Encoding
        filtered_svm_df[qkey] = filtered_svm_df[qkey].apply(lambda x: encode_expectation(x))
        filtered_wvma_df[qkey] = filtered_wvma_df[qkey].apply(lambda x: encode_expectation(x))

        # SVM tally
        counts = filtered_svm_df[qkey].value_counts(dropna=False)
        for key in counts.keys():
            svm_counts[i-1][key+1] += counts[key] # question index is 1-based; keys range from -1 to 4
        counts = svm_counts[i-1][1:] # counts of 0, 1, 2, 3, and 4
        svm_num_responses = np.sum(counts)
        svm_mean = (0*counts[0] + 1*counts[1] + 2*counts[2] + 3*counts[3] + 4*counts[4]) / svm_num_responses

        # WVMA tally
        counts = filtered_wvma_df[qkey].value_counts(dropna=False)
        for key in counts.keys():
            wvma_counts[i-1][key+1] += counts[key]
        counts = wvma_counts[i-1][1:] # counts of 0, 1, 2, 3, and 4
        wvma_num_responses = np.sum(counts)
        wvma_mean = (0*counts[0] + 1*counts[1] + 2*counts[2] + 3*counts[3] + 4*counts[4]) / wvma_num_responses
        
        # Get data
        svm_data = list(filtered_svm_df[qkey])
        wvma_data = list(filtered_wvma_df[qkey])

        # Remove empty values from data
        svm_data = [x for x in svm_data if x != -1]
        wvma_data = [x for x in wvma_data if x != -1]

        assert(svm_num_responses == len(svm_data))
        assert(wvma_num_responses == len(wvma_data))

        # compare samples
        stat, p = kruskal(svm_data, wvma_data)

        # Determine significance
        if p > alpha:
            sig = ""
        else:
            sig = "*"
            sig_count += 1

        # Cache for pooled data
        svm_pooled.extend(svm_data)
        wvma_pooled.extend(wvma_data)

        # Cache for table of results
        row = [qstring] + list(svm_counts[i-1]) + [svm_mean, svm_num_responses] + list(wvma_counts[i-1]) + [wvma_mean, wvma_num_responses, svm_mean-wvma_mean, stat, p, sig]
        rows.append(row)

    # Assemble table of results
    table = pd.DataFrame(rows, columns=["Subquestion", "SVM: empty", "SVM: 0", "SVM: 1", "SVM: 2", "SVM: 3", "SVM: 4", "SVM: avg", "SVM: num responses", "WVMA: empty", "WVMA: 0", "WVMA: 1", "WVMA: 2", "WVMA: 3", "WVMA: 4", "WVMA: avg", "WVMA: num responses", "Diff Mean (svm-wvma)", "stat", "pval", "sig"])

    # Apply Kruskal test to pooled data
    pooled_stat, pooled_p = kruskal(svm_pooled, wvma_pooled)
    pooled_diff_mean = np.mean(svm_pooled) - np.mean(wvma_pooled)

    # Print
    if verbose == True:
        print('Pooled Q%s: stat=%.3f, p=%.2e, diff_mean (svm-wvma)=%.3f, sig_subq=%s/%s' % (question_number, pooled_stat, pooled_p, pooled_diff_mean, sig_count, n_subquestions))

    return table, (pooled_stat, pooled_p, pooled_diff_mean), svm_pooled, wvma_pooled, sig_count


In [9]:
# Filter dataframe to only companion animal respondants (may have responded to other species too)
ca_svm = svm[svm['Q1'].str.contains('Companion Animal (canine and/or feline)', na=False, regex=False)].copy()

In [10]:
# Filter dataframe to only companion animal respondants (may have responded to other species too)
ca_wvma = wvma[wvma['Q1'].str.contains('Companion Animal (canine and/or feline)', na=False, regex=False)].copy()

The above code prints a question-level summary and returns a table of details for particular sub-questions.

In [11]:
table, subq_pooled_result, svm_data, wvma_data, sig_count = analyze_question(13, ca_svm, ca_wvma, n_subquestions=11)

Pooled Q13: stat=93.242, p=4.63e-22, diff_mean (svm-wvma)=0.426, sig_subq=10/11


In [12]:
table

,Subquestion,SVM: empty,SVM: 0,SVM: 1,SVM: 2,SVM: 3,SVM: 4,SVM: avg,SVM: num responses,WVMA: empty,WVMA: 0,WVMA: 1,WVMA: 2,WVMA: 3,WVMA: 4,WVMA: avg,WVMA: num responses,Diff Mean (svm-wvma),stat,pval,sig
0,Documents medical records to fulfill professi...,11,0,0,0,1,34,3.971429,35,33,0,3,1,14,64,3.695122,82,0.276307,6.579367,0.010317,*
1,Discuss recommended vaccination or preventive...,11,0,0,0,1,34,3.971429,35,33,2,2,3,12,63,3.609756,82,0.361672,7.191512,0.007325,*
2,Discuss recommended treatment plan and option...,11,0,0,0,2,33,3.942857,35,33,1,0,3,21,57,3.621951,82,0.320906,8.523766,0.003505,*
3,Discuss risks of recommended treatments and p...,12,0,0,0,3,31,3.911765,34,33,1,2,2,29,48,3.475610,82,0.436155,11.865805,0.000572,*
4,Acknowledge client’s knowledge level and appr...,11,0,0,0,2,33,3.942857,35,34,0,2,4,21,54,3.567901,81,0.374956,10.022894,0.001546,*
5,Discuss quality of life issues with owners,11,0,0,0,2,33,3.942857,35,33,1,3,2,22,54,3.524390,82,0.418467,10.466737,0.001215,*
6,Engages clients in difficult conversations su...,11,0,0,1,7,27,3.742857,35,33,2,8,3,33,36,3.134146,82,0.608711,11.513637,0.000691,*
7,Engage with clients and co,11,0,0,0,1,34,3.971429,35,34,0,2,1,12,66,3.753086,81,0.218342,5.048568,0.024646,*
8,Engage co,12,0,0,1,11,22,3.617647,34,33,8,7,13,25,29,2.731707,82,0.885940,12.973783,0.000316,*
9,"Elicit client goals, expectations, perspectiv...",11,0,0,0,2,33,3.942857,35,33,2,2,4,28,46,3.390244,82,0.552613,16.220666,0.000056,*


## Companion Animal Group

Recall that the companion animal group contains a set of questions. Now let's use the modularized question analysis code to perform a similar analysis across all questions in the companion animal group.

In [13]:
# cache data across all groups
group_data = []
group_columns = ["Group", "Pooled stat", "Pooled p", "Pooled diff_mean (svm-wvma)", "Num questions", "Fraction of sig questions", "Pooled num SVM reponses", "Pooled num WVMA responses"]

In [14]:
# cache tables
output_tables = []
output_tables_sheet_names = []

# cache subquestion table data
output_subq_data = []

In [15]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']

assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [16]:
# Code to analyze all questions within the group

def analyze_group(question_list, n_subq_list, question_strings, filtered_svm_df, filtered_wvma_df, alpha=0.05):
    svm_pooled = [] # now pooling over entire group
    wvma_pooled = []
    rows = []
    sig_count = 0
    subq_tables = []
    subq_tables_names = []

    for i in range(len(question_list)):
        question_number = question_list[i]
        n_subquestions = n_subq_list[i]
        question_string = question_strings[i]

        # Run analysis
        table, subq_pooled_result, svm_data, wvma_data, sig_subq = analyze_question(question_number, filtered_svm_df, filtered_wvma_df, n_subquestions, verbose=False)
        pooled_stat, pooled_p, pooled_diff_mean = subq_pooled_result
        svm_num_responses = len(svm_data)
        wvma_num_responses = len(wvma_data)

        # Cache procedure tables
        subq_tables.append(table)
        subq_tables_names.append('Q'+str(question_number))

        # Pool
        svm_pooled.extend(svm_data)
        wvma_pooled.extend(wvma_data)

        # Determine significance
        if pooled_p > alpha:
            sig = ""
        else:
            sig = "*"
            sig_count += 1

        # Cache data for group summary
        row = ['Q'+str(question_number), question_string, pooled_stat, pooled_p, sig, pooled_diff_mean, n_subquestions, "%i/%i" % (sig_subq,n_subquestions), svm_num_responses, wvma_num_responses]
        rows.append(row)

    # Assemble table of results
    group_table = pd.DataFrame(rows, columns=["Question number", "Category", "Pooled stat", "Pooled p", "Sig", "Pooled Diff Mean (svm-wvma)", "Num subquestions", "Fraction of sig subquestions", "Pooled num SVM responses", "Pooled num WVMA responses"])                     

    # Apply Kruskal test to pooled data
    pooled_stat, pooled_p = kruskal(svm_pooled, wvma_pooled)
    pooled_diff_mean = np.mean(svm_pooled) - np.mean(wvma_pooled)

    # Print
    print('Group result (all questions): stat=%.3f, p=%.2e, diff_mean (svm-wvma)=%.3f, sig_subq=%s/%s' % (pooled_stat, pooled_p, pooled_diff_mean, sig_count, len(question_list)))

    return group_table, (pooled_stat, pooled_p, pooled_diff_mean), sig_count, len(question_list), len(svm_pooled), len(wvma_pooled), (subq_tables, subq_tables_names)

In [17]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, ca_svm, ca_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Companion Animal", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Companion Animal")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=116.286, p=4.12e-27, diff_mean (svm-wvma)=0.423, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num SVM responses,Pooled num WVMA responses
0,Q13,Communication practices,93.241942,4.627699e-22,*,0.425950,11,10/11,383,900
1,Q14,Professional and business practices,42.826791,5.980745e-11,*,0.680261,6,3/6,210,496
2,Q15,Ethics and professional practices,8.707308,3.169369e-03,*,0.222125,8,2/8,280,656


## Special Species Group

In [18]:
# Filter dataframes to only companion animal respondants (may have responded to other species too)
ss_svm = svm[svm['Q1'].str.contains('Special Species', na=False, regex=False)].copy()
ss_wvma = wvma[wvma['Q1'].str.contains('Special Species', na=False, regex=False)].copy()

In [19]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']

assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [20]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, ss_svm, ss_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Special Species", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Special Species")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=88.072, p=6.31e-21, diff_mean (svm-wvma)=0.829, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num SVM responses,Pooled num WVMA responses
0,Q13,Communication practices,42.081740,8.753678e-11,*,0.665584,11,8/11,132,154
1,Q14,Professional and business practices,42.179853,8.325331e-11,*,1.622490,6,5/6,72,83
2,Q15,Ethics and professional practices,19.775269,8.710245e-06,*,0.470238,8,1/8,96,112


## Food Animal Group

In [21]:
# Filter dataframes to only companion animal respondants (may have responded to other species too)
fa_svm = svm[svm['Q1'].str.contains('Food Animal', na=False, regex=False)].copy()
fa_wvma = wvma[wvma['Q1'].str.contains('Food Animal', na=False, regex=False)].copy()

In [22]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']


assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [23]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, fa_svm, fa_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Food Animal", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Food Animal")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=38.415, p=5.72e-10, diff_mean (svm-wvma)=0.344, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num SVM responses,Pooled num WVMA responses
0,Q13,Communication practices,24.879436,6.102997e-07,*,0.356345,11,2/11,132,373
1,Q14,Professional and business practices,12.030221,5.234487e-04,*,0.536765,6,2/6,72,204
2,Q15,Ethics and professional practices,6.567898,1.038345e-02,*,0.181373,8,1/8,96,272


## Equine Group

In [24]:
# Filter dataframes to only companion animal respondants (may have responded to other species too)
eq_svm = svm[svm['Q1'].str.contains('Equine', na=False, regex=False)].copy()
eq_wvma = wvma[wvma['Q1'].str.contains('Equine', na=False, regex=False)].copy()

In [25]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']


assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [26]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, eq_svm, eq_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Equine", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Equine")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=34.403, p=4.48e-09, diff_mean (svm-wvma)=0.329, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num SVM responses,Pooled num WVMA responses
0,Q13,Communication practices,16.261885,0.000055,*,0.311067,11,1/11,110,253
1,Q14,Professional and business practices,10.391739,0.001266,*,0.544928,6,1/6,60,138
2,Q15,Ethics and professional practices,10.235301,0.001378,*,0.191848,8,1/8,80,184


## Group Summary

In [27]:
group_summary_table = pd.DataFrame(group_data, columns=group_columns)

In [28]:
pvals = list(group_summary_table['Pooled p'])
ALPHA = 0.05

sigs = []
for p in pvals:
  if p > ALPHA:
      sig = ""
  else:
      sig = "*"
  sigs.append(sig)

group_summary_table.insert(loc=3, column='Sig', value=sigs)

In [29]:
# Add group summary to the beginning of output tables
output_tables.insert(0, group_summary_table)
output_tables_sheet_names.insert(0, "Group summary")

In [30]:
group_summary_table

,Group,Pooled stat,Pooled p,Sig,Pooled diff_mean (svm-wvma),Num questions,Fraction of sig questions,Pooled num SVM reponses,Pooled num WVMA responses
0,Companion Animal,116.285604,4.115488e-27,*,0.422896,3,3/3,873,2052
1,Special Species,88.071722,6.312362e-21,*,0.829351,3,3/3,300,349
2,Food Animal,38.414693,5.720013e-10,*,0.343863,3,3/3,300,849
3,Equine,34.403083,4.480131e-09,*,0.329043,3,3/3,250,575


Looking at pooled comparisons at the group level, there are significant differences between the SVM and WVMA expectations for all species groups.

As we saw, there are significant differences at the individual question and subquestion levels within all groups as well.

# Generate tables

We will generate the following tables using pooled data from these experiments:

1.   `summary_nontechnical.xlsx`: Group summary table and a table for procedure sets (questions) within each group. This summary focuses on the differences between species areas.
2.   `companion_animal_nontechnical.xlsx`: Tables for all procedures within the companion animal group.
3.   `special_species_nontechnical.xlsx`: Tables for all procedures within the special species group.
4.   `food_animal_nontechnical.xlsx`: Tables for all procedures within the food animal group.
5.   `equine_nontechnical.xlsx`:Tables for all procedures within the equine group.
6. `summary_nontechnical_allspecies.xlsx`: Summary table for responses to procedures (subquestions) pooled across species areas.



## Summary

In [31]:
writer = pd.ExcelWriter('summary_nontechnical.xlsx', engine='xlsxwriter')

for i,table in enumerate(output_tables):
    sheet_name = output_tables_sheet_names[i]
    table.to_excel(writer, sheet_name=sheet_name, index=False)

    # Auto-adjust columns widths
    for column in table:
        column_width = max(table[column].astype(str).map(len).max(), len(column))
        col_idx = table.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

writer.save()

## All Species Summary

In [32]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, svm, wvma)

Group result (all questions): stat=138.341, p=6.14e-32, diff_mean (svm-wvma)=0.391, sig_subq=3/3


In [33]:
subq_tables, subq_tables_names = subq_data

# Loop through tables
writer = pd.ExcelWriter('summary_nontechnical_allspecies.xlsx', engine='xlsxwriter')

for i,table in enumerate(subq_tables):
    sheet_name = subq_tables_names[i]
    table.to_excel(writer, sheet_name=sheet_name, index=False)

    # Auto-adjust columns widths
    for column in table:
        column_width = max(table[column].astype(str).map(len).max(), len(column))
        col_idx = table.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

writer.save()

## All procedures

In [34]:
for i, file in enumerate(['companion_animal_nontechnical.xlsx', 'special_species_nontechnical.xlsx', 'food_animal_nontechnical.xlsx', 'equine_nontechnical.xlsx']):
    subq_data = output_subq_data[i]
    subq_tables, subq_tables_names = subq_data

    # Loop through tables
    writer = pd.ExcelWriter(file, engine='xlsxwriter')

    for i,table in enumerate(subq_tables):
        sheet_name = subq_tables_names[i]
        table.to_excel(writer, sheet_name=sheet_name, index=False)

        # Auto-adjust columns widths
        for column in table:
            column_width = max(table[column].astype(str).map(len).max(), len(column))
            col_idx = table.columns.get_loc(column)
            writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

    writer.save()